# Session 11: Advanced Retrieval with LangChain

## Learning Objectives:

- Understand and implement multiple retrieval strategies for RAG
- Compare naive, BM25, multi-query, parent-document, contextual compression, ensemble, and semantic chunking approaches
- Build RAG chains over a health and wellness knowledge base using LangChain and QDrant

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

---

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

> NOTE: Create a `.env` file in this directory with `OPENAI_API_KEY` and `COHERE_API_KEY` to avoid being prompted each time.

In [1]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [2]:
if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [3]:
# Setup Langsmith:
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")


if not os.environ.get("LANGCHAIN_PROJECT"):
    os.environ["LANGCHAIN_PROJECT"] = "AIM-Advanced-Retrieval"
print(os.environ["LANGCHAIN_PROJECT"])


AIM-Advanced-Retrieval


## Task 2: Data Collection and Preparation

We'll be using our Health and Wellness Guide - a comprehensive resource covering exercise, nutrition, sleep, stress management, habits, and common health concerns.

### Data Preparation

We'll load the wellness guide as a single document, then split it into smaller chunks using a `RecursiveCharacterTextSplitter` for our vector store. We also keep the raw (unsplit) document for use with the Parent Document Retriever and Semantic Chunker later.

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("data/HealthWellnessGuide.txt")
raw_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
wellness_docs = text_splitter.split_documents(raw_docs)

Let's verify our data was loaded and split correctly!

In [5]:
print(f"Raw documents: {len(raw_docs)}")
print(f"Split chunks: {len(wellness_docs)}")
print(f"\nExample chunk:\n{wellness_docs[0]}")

Raw documents: 1
Split chunks: 45

Example chunk:
page_content='The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND MOVEMENT

Chapter 1: Understanding Exercise Basics

Exercise is one of the most important things you can do for your health. Regular physical activity can improve your brain health, help manage weight, reduce the risk of disease, strengthen bones and muscles, and improve your ability to do everyday activities.' metadata={'source': 'data/HealthWellnessGuide.txt'}


## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "wellness_guide".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [6]:
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = QdrantVectorStore.from_documents(
    wellness_docs,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide",
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [7]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [8]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [9]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [10]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [11]:
naive_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees; alternate arching your back up (cat) and letting it sag down (cow). Repeat 10-15 times.\n\n- **Bird Dog:** From hands and knees, extend opposite arm and leg, keep your core engaged, hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n\n- **Pelvic Tilts:** Lie on your back with knees bent; flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds and repeat 8-12 times.\n\n- **Partial Crunches:** Lie on your back with knees bent, cross arms over your chest, tighten stomach muscles, and raise shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat, hold for 15-30 seconds, then switch legs.\n\nRemember to perform these exercises gently and consult with a healthcare professional

In [12]:
naive_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in maintaining overall health. According to the provided information, sleep is essential for physical health, mental well-being, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adequate sleep (7-9 hours per night) is also important for strengthening the immune system, managing stress, and promoting better mental health. Additionally, good sleep hygiene practices and creating an optimal sleep environment can enhance sleep quality, leading to improved health outcomes.'

In [13]:
naive_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Drinking water to stay hydrated\n- Applying cold or warm compresses to the head or neck\n- Resting in a dark, quiet room\n- Gentle massage of the temples and neck\n- Using essential oils such as peppermint or lavender\n- Practicing deep breathing, progressive muscle relaxation, or grounding techniques\n- Taking short walks, preferably outdoors or in nature\n- Listening to calming music\n\nThese approaches can help alleviate stress and reduce headache symptoms naturally.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [14]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(wellness_docs)

We'll construct the same chain - only changing the retriever.

In [15]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [16]:
bm25_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help alleviate lower back pain include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees. Alternate between arching your back up (cat) and letting it sag down (cow). Perform 10-15 repetitions.\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold each extension for about 5 seconds, then switch sides. Do 10 repetitions on each side.\n- **Pelvic Tilts:** Lie on your back with knees bent. Flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds and repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can help relieve lower back discomfort and prevent future issues.'

In [17]:
bm25_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a crucial role in overall health by promoting restorative rest that supports physical and mental well-being. Proper sleep hygiene, such as maintaining a consistent sleep schedule, creating a comfortable sleep environment, and establishing relaxing bedtime routines, helps improve sleep quality. Adequate sleep is essential for immune function, mental health, and proper nutrient absorption. Conversely, poor sleep or insomnia can negatively impact health, leading to issues like fatigue, weakened immunity, and mental health challenges. Therefore, prioritizing good sleep practices is important for maintaining overall wellness.'

In [18]:
bm25_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include relaxation techniques such as deep breathing and progressive muscle relaxation, practicing meditation, and using herbal teas like chamomile or valerian root. Additionally, ensuring proper hydration, good sleep habits, and managing triggers like eye strain or skipped meals can help reduce headaches related to stress.'

It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

##### Answer:

Query: "How does REM help the brain?"

`REM` is a keyword and with BM25 we would match it exactly. This is important because we are looking for the exact match of the technical term, not a semantic approximation that the embeddings would give us.

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [19]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [20]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [21]:
contextual_compression_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees, then alternate between arching your back upwards (cat) and letting it sag downwards (cow). Perform 10-15 repetitions.\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for about 5 seconds, then switch sides. Do 10 repetitions on each side.\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening your abdominal muscles and tilting your pelvis slightly. Hold for 10 seconds and repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can help alleviate lower back discomfort and prevent future issues.'

In [22]:
contextual_compression_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. It is essential for physical health, mental well-being, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that help regulate growth and appetite. Getting enough quality sleep, typically 7-9 hours per night for adults, supports these processes and helps maintain overall health. Factors such as creating a comfortable sleep environment can improve sleep quality, further benefiting overall health.'

In [23]:
contextual_compression_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Drinking water to stay hydrated\n- Applying a cold or warm compress to the head or neck\n- Resting in a dark, quiet room\n- Gently massaging the temples and neck\n- Using peppermint or lavender essential oils\n- Maintaining a regular sleep schedule\n- Practicing deep breathing exercises\n- Trying progressive muscle relaxation\n- Using grounding techniques, such as identifying sensory experiences\n- Taking short walks, especially in nature\n- Listening to calming music\n\nThese methods can help alleviate stress and reduce headache symptoms naturally.'

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [24]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
) 

In [25]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [26]:
multi_query_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- Cat-Cow Stretch: Start on your hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Perform 10-15 repetitions.\n- Bird Dog: From hands and knees, extend the opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles, and lift shoulders off the floor. Repeat 8-12 times.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds and repeat 8-12 times.\n\nThese exercises are gentle and targeted to strengthen and stretch the muscles supporting the lower back, potentially all

In [27]:
multi_query_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. It is essential for physical health, mental well-being, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adequate sleep—typically 7 to 9 hours per night—is linked to a stronger immune system, better recovery from illness, and improved mood and mental health. Poor sleep or sleep disorders like insomnia can lead to issues such as increased stress, weakened immune function, difficulty concentrating, and higher risk of chronic conditions. Therefore, maintaining good sleep hygiene and a regular sleep routine are important for promoting overall health and well-being.'

In [28]:
multi_query_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Deep breathing exercises (e.g., inhale for 4 counts, hold for 4, exhale for 4)\n- Progressive muscle relaxation (tensing and releasing muscle groups)\n- Grounding techniques (naming things you see, hear, feel, smell, and taste)\n- Taking a short walk, preferably in nature\n- Listening to calming music\n- Drinking water to stay hydrated\n- Applying cold or warm compresses to the head or neck\n- Resting in a dark, quiet room\n- Gentle massage of the temples and neck\n- Using essential oils such as peppermint or lavender\n- Maintaining a regular sleep schedule\n- Practicing mindfulness and meditation to reduce overall stress\n\nThese methods can help alleviate headache symptoms and promote relaxation.'

### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

##### Answer:

Typical user questions are vague or short. So they "hide" a bunch of the context or make it implicit. By reformulating a query into multiple variants we can highlight certain parts of the query that are not stated out explicitly. The reformulation adds detail to the query and extra information that it thinks it's relevant. So when we do the actual retrieval our query's embedding is more "precise" so if we use a similarity metric to find relevant chunks we can come "closer" to more chunks that way, thus increasing recall. One way to think about it is that instead of throwing a blunt object at the center of a tank to catch a specific type of fish, we instead shoot multiple precise arrows in targetted spots in the tank. We will catch more fish of the type we want and less fish that are "collateral". 

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. We split the full document into large "parent" chunks (e.g. 2000 characters).
2. Each parent chunk is further split into smaller "child" chunks (e.g. 400 characters).
3. The child chunks are stored in a VectorStore, while the parent chunks are stored in an in-memory docstore.
4. When we query our Retriever, we do a similarity search comparing our query vector to the child chunks.
5. Instead of returning the child chunks, we return their associated parent chunks.

The basic idea is:

- **Search** for small, focused chunks (better semantic matching)
- **Return** big chunks (richer surrounding context)

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by defining our parent and child splitters.

In [29]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [30]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="wellness_parent_child",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="wellness_parent_child", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [31]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore=parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [32]:
parent_document_retriever.add_documents(raw_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [33]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [34]:
parent_document_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

"To help with lower back pain, gentle stretching and strengthening exercises are recommended. Some effective exercises include:\n\n- **Cat-Cow Stretch:** On hands and knees, alternate arching your back up (cat) and letting it sag down (cow). Perform 10-15 repetitions.\n- **Bird Dog:** On hands and knees, extend opposite arm and leg, hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- **Partial Crunches:** Lie on your back with knees bent, cross arms over chest, tighten your stomach muscles, and lift your shoulders off the floor. Do 8-12 repetitions.\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat, hold for 15-30 seconds, then switch legs.\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds, repeating 8-12 times.\n\nThese exercises can help alleviate lower back pain and prevent future ep

In [35]:
parent_document_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep significantly affects overall health by supporting physical, mental, and cognitive functions. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adequate sleep, typically 7-9 hours per night for adults, helps maintain immune function, mental well-being, and emotional stability. Poor sleep or sleep disorders like insomnia can lead to fatigue, decreased cognitive performance, weakened immune response, and increased risk for various health issues. Practicing good sleep hygiene and creating a healthy sleep environment are essential for achieving quality rest and supporting overall health.'

In [36]:
parent_document_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include deep breathing exercises, gentle massage, practicing mindfulness or meditation, and engaging in relaxing activities like listening to calming music or taking a short walk in nature. To help manage headaches specifically, staying hydrated, applying cold or warm compresses to the head or neck, resting in a dark, quiet room, and using essential oils such as peppermint or lavender can be effective.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [37]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [38]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [39]:
ensemble_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting the pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.\n- Partial Crunches: Lie on your back with knees bent, cross arms over your chest, tighten your stomach muscles, and raise your shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n\nThese exercises are gentle stretching and strengthening activities that can alleviate discomfo

In [40]:
ensemble_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in overall health by supporting physical, mental, and cognitive functions. During sleep, the body repairs tissues, consolidates memories, and regulates hormones that influence growth and appetite. Adequate sleep, typically 7-9 hours per night, helps strengthen the immune system, improve mood, enhance focus, and maintain hormonal balance. Poor or insufficient sleep can lead to health problems such as weakened immunity, increased stress, and cognitive impairments. Therefore, maintaining good sleep hygiene and creating an optimal sleep environment are essential for promoting overall wellness.'

In [41]:
ensemble_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\nFor Headaches:\n- Drink water and stay hydrated\n- Apply cold or warm compresses to the head or neck\n- Rest in a dark, quiet room\n- Gentle massage of the temples and neck\n- Use peppermint or lavender essential oils\n- Maintain a regular sleep schedule\n- Manage triggers such as dehydration, eye strain, and certain foods\n\nFor Stress:\n- Practice deep breathing exercises (e.g., inhale for 4 counts, hold for 4, exhale for 4)\n- Use progressive muscle relaxation, tensing and releasing muscle groups\n- Engage in grounding techniques by naming things you see, hear, feel, smell, and taste\n- Take short walks, especially in nature\n- Listen to calming music\n- Maintain a consistent evening wind-down routine with activities like light stretching, reading, or warm baths\n- Practice mindfulness and meditation to stay present and reduce stress\n\nThese natural approaches can help manage both stress and headaches effectively.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [42]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [43]:
semantic_documents = semantic_chunker.split_documents(raw_docs)

Let's create a new vector store.

In [44]:
semantic_vectorstore = QdrantVectorStore.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide_semantic_chunks"
)

We'll use naive retrieval for this example.

In [45]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [46]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [47]:
semantic_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles, and raise shoulders off the floor. Do 8-12 repetitions.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening abs and tilting your pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises are recommended to alleviate lower back discomfort and help prevent future episodes.'

In [48]:
semantic_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in overall health by supporting physical, mental, and emotional well-being. During sleep, the body repairs tissues, consolidates memories, and releases crucial hormones that regulate growth and appetite. Adequate sleep—typically 7-9 hours for adults—also helps maintain a healthy immune system, manage stress, improve cognitive function, and support emotional resilience. Poor sleep quality or insufficient sleep can lead to increased risk of health problems such as weakened immunity, mental health issues, weight gain, and chronic conditions. Therefore, prioritizing good sleep hygiene and creating a conducive sleep environment are essential for maintaining overall health.'

In [49]:
semantic_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Drinking plenty of water to stay hydrated\n- Applying cold or warm compresses to the head or neck\n- Resting in a dark, quiet room\n- Gentle massage of temples and neck muscles\n- Using essential oils like peppermint or lavender\n- Practicing relaxation techniques such as deep breathing exercises or progressive muscle relaxation\n- Engaging in short walks, especially in nature\n- Listening to calming music\n\nThese methods can help alleviate headaches and reduce stress naturally.'

### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

##### Answer:

With short and highly repetitive sentences we will have similarity scores that are close to each other. A `percentage` threshold will become very sensitive to the variation. Imagine all our values are between 0.88 and 0.90 similarity scores. To pick the default 95-percentile would mean a value of around 0.899 or something similar but then it will depend on the distribution of the values. If most values are close to the 0.899 then we would have very few chunks. If they are below 0.899 we will have too many chunks. Standard Deviation will also have a similar problem and `interquartile` as well since those are designed to detect outliers. One interesting approach is to leverage `gradient` since that looks at the derivate (the difference between similarities) and this gives us more variability because it eliminates the absolute magnitude of the similarity value (in our case that 0.88 goes away) so we are looking at numbers such as 0.01 and 0.02, etc. With the right threshold value there we could split between a 0.01 and 0.02. The problem is that we can't guess a priori whether a good gradient value is 0.5 or 0.015. They are very different. 

One approach would be to normalize the gradients and bring them into the [0,1] range in which case we can split on the 50% mark, but that means we'd need to modify the `SemanticChunker` source code to add a new threshold type called something like `gradient_normalized`. Another approach without modifying the implementation of `SemanticChunker` would be to use `gradient` and set an `expected_number_of_chunks` value. Then do a binary search on the actual threshold value until we come close to the expected number of docs. For example if the value of the gradient that splits our content into 10 chunks is 0.015 we would do a binary search starting with 0.5 and then searching "downwards" to 0.25 -> 0.125 ->... (using O(log) trials).

---

# 🤝 Breakout Room Part #2

### 🏗️ Activity #1:

Your task is to evaluate the various Retriever methods against each other.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparison between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [50]:
### Let's create a dataset with RAGAS
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from ragas.testset import TestsetGenerator

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
# We use the already loaded Wellness Guide to create a synthetic dataset.
dataset = generator.generate_with_langchain_docs(raw_docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/3 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/7 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [51]:
### Let's see our dataset.
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is NUTRITION AND DIET about?,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,PART 2: NUTRITION AND DIET covers the fundamen...,single_hop_specifc_query_synthesizer
1,Water is important how for health?,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,Water is essential for virtually every bodily ...,single_hop_specifc_query_synthesizer
2,What is Chapter 11 about?,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,Chapter 11 discusses stress reduction techniqu...,single_hop_specifc_query_synthesizer
3,What is sleep hygeine and how does it help imp...,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,Sleep hygiene refers to habits and practices t...,single_hop_specifc_query_synthesizer
4,How can habit formation help improve your over...,[13: The Science of Habit Formation Habits are...,Understanding how habits form can help you bui...,single_hop_specifc_query_synthesizer
5,How does understanding the components of the h...,[13: The Science of Habit Formation Habits are...,"Understanding how habits form, including the c...",single_hop_specifc_query_synthesizer
6,How does supporting immune function through li...,[13: The Science of Habit Formation Habits are...,Supporting immune function through lifestyle f...,single_hop_specifc_query_synthesizer
7,What is the APPENDIX about in the context of w...,[13: The Science of Habit Formation Habits are...,The APPENDIX provides quick reference guides f...,single_hop_specifc_query_synthesizer
8,Wha is the importnce of Monay in holistc health?,[The Personal Wellness Guide A Comprehensive R...,The context does not explicitly mention 'Monda...,single_hop_specifc_query_synthesizer
9,What does Wednesday typically signify in a wel...,[The Personal Wellness Guide A Comprehensive R...,Wednesday is designated as a rest day or a day...,single_hop_specifc_query_synthesizer


In [57]:
### Let's create a dataset with RAGAS
import time
import copy
from langsmith.evaluation import evaluate

from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig, EvaluationDataset
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
run_config = RunConfig(timeout=360)

all_retrieval_chains = {
    "naive_retrieval_chain": naive_retrieval_chain.with_config({"run_name" : "naive_retrieval_chain"}),
    "bm25_retrieval_chain": bm25_retrieval_chain.with_config({"run_name" : "bm25_retrieval_chain"}),
    "contextual_compression_retrieval_chain": contextual_compression_retrieval_chain.with_config({"run_name" : "contextual_compression_retrieval_chain"}),
    "multi_query_retrieval_chain": multi_query_retrieval_chain.with_config({"run_name" : "multi_query_retrieval_chain"}),
    "parent_document_retrieval_chain": parent_document_retrieval_chain.with_config({"run_name" : "parent_document_retrieval_chain"}),
    "ensemble_retrieval_chain": ensemble_retrieval_chain.with_config({"run_name" : "ensemble_retrieval_chain"}),
    "semantic_retrieval_chain": semantic_retrieval_chain.with_config({"run_name" : "semantic_retrieval_chain"}),
}

def evaluate_retrieval_chain(retrieval_chain):
    test_dataset = copy.deepcopy(dataset)
    for test_row in test_dataset:
        response = retrieval_chain.invoke({"question" : test_row.eval_sample.user_input})
        test_row.eval_sample.response = response["response"].content
        test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
        time.sleep(2) # To try to avoid rate limiting.

    evaluation_dataset = EvaluationDataset.from_pandas(test_dataset.to_pandas())
    return evaluate(
        evaluation_dataset,
        metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall()],
        llm=evaluator_llm,
        run_config=run_config
    )

## Let's do a dry run with one retriever.


dry_run_result = evaluate_retrieval_chain(naive_retrieval_chain.with_config({"run_name" : "dry_run"}))
dry_run_result

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

{'context_recall': 0.7222, 'faithfulness': 0.7426, 'factual_correctness': 0.5258, 'answer_relevancy': 0.9570, 'context_entity_recall': 0.3653}

In [58]:
import pandas as pd

results = []
for name, retriever in all_retrieval_chains.items():
    scores = evaluate_retrieval_chain(retriever)
    results.append({
        "retriever": name,
        "scores": scores
    })

summary_df = pd.DataFrame(results).set_index("retriever")



Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [68]:
#summary_df
summary_df["scores"].apply(lambda x: pd.DataFrame(x.scores).mean()).round(3)

,context_recall,faithfulness,factual_correctness,answer_relevancy,context_entity_recall
retriever,,,,,
naive_retrieval_chain,0.833,0.663,0.562,0.970,0.371
bm25_retrieval_chain,0.340,0.599,0.303,0.565,0.081
contextual_compression_retrieval_chain,0.722,0.737,0.499,0.974,0.289
multi_query_retrieval_chain,0.722,0.695,0.539,0.963,0.335
parent_document_retrieval_chain,0.833,0.736,0.559,0.875,0.234
ensemble_retrieval_chain,0.889,0.720,0.564,0.949,0.284
semantic_retrieval_chain,1.000,0.784,0.631,0.952,0.179


These as some averaged numbers from the Langsmith traces:

| Retriever              | Latency | Tokens |
|------------------------|---------|--------|
| naive                  | 1.3s    | 1,500  |
| bm25                   | 0.6s    | 500    |
| contextual_compression | 1.2s    | 600    |
| multi_query            | 2.5s    | 2,200  |
| parent_document        | 1.3s    | 1,300  |
| ensemble               | 3.5s    | 3,500  |
| semantic               | 1.5s    | 4,100  |

### Observations:

* `semantic_retrieval_chain` seems to have the highest relevance/recall/correctness etc. scores but it's also the most expensive. Latency is on the lower side.
* `ensemble_retrieval_chain` scores also high in all metrics but it's also the slowest and second most expensive. It is expected given how we run multiple retrievals in parallel.
* `parent_document_retrieval_chain` scores close to the ensemble with 1/3 of the latency and cost so that makes it a very practical option.
* `multi_query_retrieval_chain` is the best in entity recall but for the high latency and cost it doesn't really become practical.
* `contextual_compression_retrieval_chain` scores the highest in relevancy and decently on the other metrics but it's fast and very cheap so that's also a good practical option.
* `bm25_retrieval_chain` has very low scores (the lowest) so even though it's the fastest and cheapest it's not a good fit for this content.
* `naive_retrieval_chain` is actually not too bad compared to parent_document or multi_query so that's also a practical option.

If I had to pick a reasonable option for this dataset I'd look between the `contextual_compression` and the `parent_document` since they seem to grab more relevant context at the same or better cost vs performance ratios.